In [2]:
import numpy as np
import pandas as pd
import json
import spacy
from tqdm import tqdm
import gensim
# import warnings; warnings.simplefilter('ignore')

import utils


In [3]:
df = pd.read_json('data/emojis.json')
print(df.shape)
df.head()

(2389, 7)


,category,definition,keywords,name,senses,shortcode,unicode
0,Miscellaneous Symbols And Pictographs -> Emoji...,"The Woman Gesturing Not OK, Type-6 emoji is a ...","[dark skin tone, hand, forbidden, gesture, wom...",woman gesturing NO: dark skin tone,{'adjectives': [{'bn:00104562a': ['Contrary to...,None,U+1F645 U+1F3FF U+200D U+2640 U+FE0F
1,Miscellaneous Symbols And Pictographs -> Emoji...,"The Female Guard, Type-6 emoji is a sequence o...","[dark skin tone, woman, guard]",woman guard: dark skin tone,"{'adjectives': [], 'verbs': [{'bn:00090041v': ...",None,U+1F482 U+1F3FF U+200D U+2640 U+FE0F
2,None,The female version of the ?? Runner emoji. The...,"[racing, running, woman, marathon]",woman running,{'adjectives': [{'bn:00109994a': ['Of advancin...,None,U+1F3C3 U+200D U+2640 U+FE0F
3,Miscellaneous Symbols And Pictographs -> Emoji...,"The Woman Doing Cartwheel, Type-3 emoji is a s...","[gymnastics, medium-light skin tone, woman, ca...",woman cartwheeling: medium-light skin tone,"{'adjectives': [], 'verbs': [{'bn:00084605v': ...",None,U+1F938 U+1F3FC U+200D U+2640 U+FE0F
4,None,The female version of the ?? Golfer emoji. The...,"[woman, golf]",woman golfing,"{'adjectives': [], 'verbs': [{'bn:00088979v': ...",None,U+1F3CC U+FE0F U+200D U+2640 U+FE0F


In [4]:
for u in df.iloc[0].unicode.split(' '):
    print(u)
    utils.display_emoji(u)
    print()

U+1F645



U+1F3FF



U+200D



U+2640



U+FE0F


In [7]:
# With Word2vec

vocab_size = 30000 # keep this reasonably small for now, we're going to be doing a search over the entire vocab
model = gensim.models.KeyedVectors.load_word2vec_format('data/GoogleNews-vectors-negative300.bin', binary=True, limit=vocab_size)

# Init blank english spacy nlp object
nlp = spacy.blank('en')

for i in tqdm(range(vocab_size)):
    nlp.vocab.set_vector(model.index2word[i], model.vectors[i])
# # Loop through range of all indexes, get words associated with each index.
# # The words in the keys list will correspond to the order of the google embed matrix
keys = [model.index2word[i] for i in range(vocab_size)]

# # Set the vectors for our nlp object to the google news vectors
nlp.vocab.vectors = spacy.vocab.Vectors(data=model.vectors, keys=keys)

100%|██████████| 30000/30000 [00:00<00:00, 45550.35it/s]


In [55]:
print(model['woman'].shape)
print(model['golf'].shape)
print((model['woman']+model['golf']).shape)
print(utils.cosine_similarity(model['woman'],model['golf']))
print(utils.cosine_similarity(model['woman'],model['woman']+model['golf']))
print(utils.cosine_similarity(model['golf'],model['woman']+model['golf']))
avg = np.mean(np.concatenate((np.reshape(np.array(model['woman']),(-1,1)),np.reshape(np.array(model['golf']),(-1,1))),axis=-1),axis =1)
print(utils.cosine_similarity(model['woman'],avg))
print(utils.cosine_similarity(model['golf'],avg))

(300,)
(300,)
(300,)
0.11305464059114456
0.695676326751709
0.7923993468284607
0.695676326751709
0.7923993468284607


In [21]:
print(nlp.vocab.strings)

### Interesting: Cosine similarity between a word and the sum of two words is the same as the similarity between the word and the vector average of the two words.

In [13]:
for word in nlp.vocab.strings:
    print(word)
    break

""


In [58]:
# With Glove.  Let's use word2vec for now.

# nlp = spacy.load('en')

# with open('data/glove.6B.100d.txt', 'r') as f:
#     for line in tqdm(f, total=400000):
#         parts = line.split()
#         word = parts[0]
#         vec = np.array([float(v) for v in parts[1:]], dtype='f')
#         nlp.vocab.set_vector(word, vec)

In [62]:
docs = [nlp(x) for x in tqdm(df.keywords.str.join(' '))]

100%|██████████| 2389/2389 [00:00<00:00, 5259.77it/s]


In [59]:
primary_unicodes = [u[0] for u in df.unicode.str.split(' ')]

In [60]:
for i in range(5):
    print(df.iloc[i]['name'])
    utils.display_emoji(primary_unicodes[i])
    print()

woman gesturing NO: dark skin tone



woman guard: dark skin tone



woman running



woman cartwheeling: medium-light skin tone



woman golfing


In [63]:
emoji_vectors = {}
for i,u in enumerate(primary_unicodes):
    emoji = chr(int(u[2:], 16))
    emoji_vectors[emoji] = docs[i].vector

In [64]:
def get_similar_emoji(word_vector, n=5):
    '''
    returns doc index of most n most similar emoji
    args:
        word_vector: vector embedding of word
    returns:
        int index of most similar emoji if n == 1
        (list) index of most similar emoji if n > 1
    '''
    indexes = np.argsort([-utils.cosine_similarity(word_vector, emoji.vector) for emoji in docs])
    return indexes[0] if n == 1 else indexes[:n]

In [65]:
v = nlp(u'star').vector
for i in get_similar_emoji(v):
    print(df.iloc[i]['name'])
    utils.display_emoji(primary_unicodes[i])
    print()

eight-pointed star


/home/sm_ferro54/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)



sparkles



white medium star



night with stars



dotted six-pointed star


In [66]:
v = nlp(u'star').vector + nlp(u'movie').vector
for i in get_similar_emoji(v):
    print(df.iloc[i]['name'])
    utils.display_emoji(primary_unicodes[i])
    print()

clapper board


/home/sm_ferro54/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)



white medium star



sparkles



eight-pointed star



dizzy


In [67]:
vocab = list(nlp.vocab.strings)

In [68]:
def get_similar_words(emoji_vector, n=5):
    '''
    returns doc index of most n most similar words
    args:
        emoji_vector: vector embedding of emoji
    returns:
        int index of most similar word if n == 1
        (list) index of most similar words if n > 1
    '''
    indexes = np.argsort([-utils.cosine_similarity(emoji_vector, nlp(word).vector) for word in vocab])[:n]
    return indexes[0] if n == 1 else indexes[:n]

In [69]:
v1 = nlp(u'star').vector
v2 = emoji_vectors['⭐']
print(utils.cosine_similarity(v1, v2))

1.0


In [70]:
v1 = nlp(u'hello').vector
v2 = emoji_vectors['⭐']
print(utils.cosine_similarity(v1, v2))

0.12529227137565613


In [71]:
print(utils.cosine_similarity(v2, nlp.vocab[u'star'].vector))

1.0


In [72]:
v = emoji_vectors['⭐']
for i in tqdm(get_similar_words(v)):
    print(i)
    print(vocab[i])
    print()

/home/sm_ferro54/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 5/5 [00:00<00:00, 1502.37it/s]

1795
star

2910
stars

10017
superstar

19634
superstars

9351
standout



In [73]:
def encode(text):
    '''
    encodes text into emoji
    args:
        (list) text
    returns:
        (list) unicodes of emoji
    '''
    vectors = [nlp(word).vector for word in text.split(' ')]
    return [primary_unicodes[get_similar_emoji(v, n=1)] for v in tqdm(vectors)] 

def decode(emoji):
    '''
    decodes emoji into text
    args:
        (list) emoji
    returns:
        (list) words
    '''
    vectors = [emoji_vectors[e] for e in emoji]
    return [vocab[get_similar_words(v, n=1)] for v in tqdm(vectors)] 

In [78]:
text = 'Woman golfing'
encoding = encode(text)
emoji = [chr(int(unicode[2:], 16)) for unicode in encoding]
for e in emoji:
    print(e, end='')

  0%|          | 0/2 [00:00<?, ?it/s]/home/sm_ferro54/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 2/2 [00:00<00:00,  7.20it/s]

♀🏌

In [79]:
decoding = decode(emoji)

  0%|          | 0/2 [00:00<?, ?it/s]/home/sm_ferro54/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
100%|██████████| 2/2 [00:09<00:00,  4.76s/it]


In [80]:
print(decoding)

['woman', 'skin']


In [76]:
utils.cosine_similarity(nlp(text).vector, nlp(' '.join(decoding)).vector)

0.19798541069030762

In [77]:
for i in get_similar_emoji(nlp('Berkeley').vector):
    print(df.iloc[i]['name'])
    utils.display_emoji(primary_unicodes[i])
    print()

woman scientist


/home/sm_ferro54/anaconda3/lib/python3.6/site-packages/scipy/spatial/distance.py:698: RuntimeWarning: invalid value encountered in float_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)



man scientist



pot of food



woman scientist: dark skin tone



woman scientist: light skin tone
